----
### *The 12th Computational Neuroscience Winter School*

# Tutorial I: Brain Inspired Computation - Based Part
----
__Date:__ Jan. 10, 2023

__Content Creator:__ Chongming Liu

# Tutorial Objectives

In this notebook, we'll learn the history of the development of artificial intelligence in the last century. We gonna to learn some classical models and algorithms and with some interesting examples as exercises.

1. MP Model
1. Hebbian Learning
1. Hopfield Model
1. Building Up A Simple Artificial Neural Network

---
# Setup


---

In [ ]:
# Import most modules and functions needed
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import numpy as np
from scipy.stats import chi2
import matplotlib as mpl
mpl.rcParams['lines.linewidth'] = 2.5
mpl.rcParams['legend.fontsize'] = 12
mpl.rcParams['axes.labelsize'] = 12
mpl.rcParams['axes.titlesize'] = 14
mpl.rcParams['font.weight'] = 'bold'
mpl.rcParams['font.size'] = 12
mpl.rcParams['axes.labelweight'] = 'bold'
import matplotlib.pyplot as plt
import sklearn.datasets
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split 
import pandas as pd

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

---
# Section 1: MP Model

*Reference:* 

*McCulloch, W.S., & Pitts, W.H. (1943). A logical calculus of the ideas immanent in nervous activity. Bulletin of Mathematical Biology, 52, 99-115.*

---

Warren S. Mcculloch and Walter Pitts (1943) found that single neuron can do logical operations. Based on this fact, they proposed a kind of computational model that performs like a neuron.

<img src="./imgs/MP_model.png" width="50%">

MP Model can be used to do binary classification (Actually, MP model can be seen as linear classifier), since the output of the model is boolean. Here we take Wiscosin dataset as an example. Wiscosin dataset is a dataset for breast cancer, which contains 30 features about the tumor, and you need to classify that which is malignant and which is benign. 


#### Wiscosin dataset

In [ ]:
breast_cancer = sklearn.datasets.load_breast_cancer()

X = breast_cancer.data
Y = breast_cancer.target
data = pd.DataFrame(breast_cancer.data,columns=breast_cancer.feature_names)
data['class'] = breast_cancer.target

data

#### Spliting the data to training set and test set

In [ ]:
X = data.drop('class',axis=1)
Y = data['class']

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.1, stratify=Y, random_state=0)


X_binarise_train = X_train.apply(pd.cut, bins=2, labels=[1,0])
X_binarise_test = X_test.apply(pd.cut, bins=2, labels=[1,0])


X_binarise_train = X_binarise_train.values
X_binarise_test = X_binarise_test.values


print('Target: ',Y.mean(), Y_train.mean(), Y_test.mean())

#### Constrcut MP neuron
 
We should note that there didn't exist an training algorithm for us to find out $w$, so here we would fix all the weight to $1$, and try to search a better $\theta$ for classification. The input-output relation becomes:
$$\hat y_i=\Theta(\sum_{j}^n x_i^j-\theta)$$
Since there are totally $n$ features, here we investigate the best integer threshold between $[0,n]$

In [ ]:
class MPNeuron:
    def __init__(self):
        self.b = None
    
    def model(self,x):
        return (sum(x) >= self.b)
    
    def predict(self,X):
        Y = []
        for x in X:
            Y.append(self.model(x))
        return np.array(Y)
    
    def fit(self,X,Y):
        accuracy = {}
        for b in range(X.shape[1] + 1):
            self.b = b
            Y_pred = self.predict(X)
            accuracy[b] = accuracy_score(Y_pred,Y)
            
        best_b = max(accuracy, key = accuracy.get)
        self.b = best_b
        
        print('Best threshold:', best_b)
        print('Best accuracy: {:.2f}'.format(accuracy[best_b]))

        
mp_neuron = MPNeuron()
mp_neuron.fit(X_binarise_train,Y_train)

w = mp_neuron.predict(X_binarise_test)
print('Test accuracy: {:.2f}'.format(accuracy_score(w,Y_test)))

---
# Section 2: Hebbian Learning

*Reference:* 

*Hebb, D.O. (1949). The Organization of Behavior: A Neuropsychological Theory.*

---

Hebb's rule: "Fire together, wire together." Consider using Hebb's rule in the MP model:

$$
y(x)=\Theta(w^Tx-\theta)
$$

then, a sequence of inputs $(x_i)_{i=1}^n$ are given to the model, the weights would updates as

$$
w_{t+1}=w_{t}+\eta y(x_i)x_i
$$

So Hebbian learning is an algorithm for unsupervised learning. If we consider a simplified MP model:

$$y(x)=w^Tx$$

And we update the weight only when the model has seen all data, then the updating rule becomes:

$$w_{t+1}=w_{t}+\eta \sum_{i=1}^ny(x_i)x_i=(I+\eta \sum_{i=1}^n x_ix_i^T)w_{t}$$

So we have

$$w_{t}=(I+\eta C)^t w_{0}$$

where $C=\sum_{i=1}^n x_ix_i^T$ is the covariance matrix of data. Then we know the weight vector would finally converge to the direction of leading eigenvector of $C$, as long as  the initial iterate has a projection onto the leading eigenvector.

In [ ]:
# generate data matrix
X = np.random.randn(10000,100)
mean_x = np.ones(100)
X_data = mean_x + X

# Compute covariance matrix
C = np.matmul(X_data.T, X_data)

w = np.random.randn(100)
# Hebbian learning
eta = 0.1
for step in range(10000):
    w += eta*np.matmul(C,w)
    w = w/np.linalg.norm(w)
    
# Compute the absolute cos between weight the mean of data
abs_cos = abs(np.matmul(w,mean_x))/(np.linalg.norm(mean_x))
print('absolute cos between weight the mean of data: {:.2f}'.format(abs_cos))

---
# Section 3: Hopfield Model

*Reference:* 

*Hopfield, J.J. (1982). Neural networks and physical systems with emergent collective computational abilities. Proceedings of the National Academy of Sciences of the United States of America, 79 8, 2554-8 .*

*Hopfield, J.J. (1984). Neurons with graded response have collective computational properties like those of two-state neurons. Proceedings of the National Academy of Sciences of the United States of America, 81 10, 3088-92 .*

*Gerstner, W., Kistler, W.M., Naud, R., & Paninski, L. (2014). Neuronal Dynamics: From Single Neurons To Networks And Models Of Cognition.*
    
---

The Hopfield model consists of a network of $N$ neurons, labeled by a lower index $i$, with $1\leq i\leq N$. Similar to MP model, neurons in the Hopfield model have only two states. A neuron $i$ is "ON" if its state variable takes the value $S_i=+1$ and "OFF" if $S_i=-1$. Neurons interact with each other with weights $w_{ij}$. The input potential of neuron $i$, influenced by the activity of other neurons is

$$h_i(t)=\sum_j w_{ij}S_{j}(t)$$

The input potential at time $t$ influences the probabilistic update of the state variable $S_i$ in the next time step:

$$P\{S_i(t+\Delta t)=+1|h_i(t)\}=g(h_i(t))=g(\sum_j w_{ij}S_{j}(t))$$

where g is a monotonically increasing gain function with values between zero and 1. If $g$ is Heaviside function, then the dynamics are therefore deterministic and summarized by the update rule

$$S_i(t+\Delta t)=sgn[h_i(t)]$$

The most powerful property of Hopfield model is that with a suitable choice of the coupling matrix $w_{ij}$,
memory items can be retrieved by the collective dynamics. If we need the network to store and recall $M$ different patterns, patterns are labeled by the index $\mu$ with $1\leq \mu\leq M$. Each pattern $\mu$ is define as desired configuration $\{p_i^\mu=\pm 1; 1\leq i\leq N\}$. The network of $N$ neurons is said to correctly represent pattern $\mu$, if the state of all neurons $1\leq u\leq N$ is $S_i(t)=S_i(t+\Delta t)=p_i^\mu$. In other words, patterns must
be fixed points of the dynamics.

We can choose the weight as:

$$w_{ij}=\frac{1}{N}\sum_{\mu=1}^{M}p_i^\mu p_j^\mu$$

Then the network stores all patterns. 

#### Math detour

Define the similarity between the current state and a pattern $\mu$:

$$m^\mu(t) = \frac{1}{N}\sum_{i}p_i^\mu S_i(t)$$

Define energy function

$$E(t)=-\sum_i\sum_j w_{ij}S_i(t)S_j(t)$$

Then the change in energy by updating the state of neuron $k$ is

$$E(t+\Delta t)-E(t)=-4h_k sgn(h_k)<0$$

And if we insert the definitionof the connection weights $w_{ij}=\frac{1}{N}\sum_{\mu=1}^{M}p_i^\mu p_j^\mu$, then we find:

$$E(t)=-N\sum_{\mu}(m^\mu(t))^2$$

The maximum value of the overlap with a fixed pattern $\nu$ is $m^\nu = 1$. Moreover, for
random patterns, the correlations between patterns are small. Therefore, if $m^{\nu} = 1$ (i.e.,
recall of pattern $\nu$) the overlap with other patterns $\mu\neq \nu$ is $m^\mu\approx 0$. Therefore, the energy landscape can be visualized with multiple minima of the same depth, each minimum corresponding to retrieval of one of the patterns

#### An example

In [ ]:
# generate 'E' pattern
pattern_E = np.array([1,1,1,1,-1,1,-1,-1,-1,-1,1,1,1,-1,-1,1,-1,-1,-1,-1,1,1,1,1,-1]).reshape(25,1)

plt.figure(figsize=(1.5,1.5))
plt.imshow(pattern_E.reshape(5,5), cmap = plt.cm.binary)

In [ ]:
# generate connection weight
weight_matrix = np.matmul(pattern_E, pattern_E.T)/25

np.random.seed(2)
# initial state
state = np.sign(np.random.randn(25))
fig,ax=plt.subplots(1,2, layout="constrained", figsize=(3,6))
ax[0].imshow(state.reshape(5,5), cmap = plt.cm.binary)

# One step for hopfield iteration
state = np.sign(np.matmul(weight_matrix, state))
ax[1].imshow(state.reshape(5,5), cmap = plt.cm.binary)

In [ ]:
# generate '7' pattern
pattern_7 = np.array([-1,-1,1,1,1,-1,-1,-1,-1,1,-1,-1,-1,-1,1,-1,-1,-1,-1,1,-1,-1,-1,-1,1]).reshape(25,1)

plt.figure(figsize=(1.5,1.5))
plt.imshow(pattern_7.reshape(5,5), cmap = plt.cm.binary)

In [ ]:
# generate connection weight
weight_matrix = (np.matmul(pattern_E, pattern_E.T)+np.matmul(pattern_7, pattern_7.T))/25

# initial state
state = np.array([1,1,-1,-1,-1,1,-1,-1,-1,-1,1,1,1,-1,-1,1,-1,-1,-1,-1,1,1,1,1,-1])
fig,ax=plt.subplots(1,2, layout="constrained", figsize=(3,6))
ax[0].imshow(state.reshape(5,5), cmap = plt.cm.binary)
ax[0].set_title('initial state')

# One step for hopfield iteration
state = np.sign(np.matmul(weight_matrix, state))
ax[1].imshow(state.reshape(5,5), cmap = plt.cm.binary)
ax[1].set_title('final state')

# another initial state
state = np.array([1,1,1,1,1,-1,-1,-1,-1,1,-1,-1,-1,-1,1,-1,-1,-1,-1,1,-1,-1,-1,-1,1])
fig,ax=plt.subplots(1,2, layout="constrained", figsize=(3,6))
ax[0].imshow(state.reshape(5,5), cmap = plt.cm.binary)

# One step for hopfield iteration
state = np.sign(np.matmul(weight_matrix, state))
ax[1].imshow(state.reshape(5,5), cmap = plt.cm.binary)

---
# Section 4: Building Up A Simple Artificial Neural Network

*Reference:* 

*Paszke, A., Gross, S., Massa, F., Lerer, A., Bradbury, J., Chanan, G., Killeen, T., Lin, Z., Gimelshein, N., Antiga, L., Desmaison, A., Köpf, A., Yang, E., DeVito, Z., Raison, M., Tejani, A., Chilamkurthy, S., Steiner, B., Fang, L., Bai, J., & Chintala, S. (2019). PyTorch: An Imperative Style, High-Performance Deep Learning Library. Neural Information Processing Systems.*

---

# Modern artificial neural network:
<img src="./imgs/simple_network.png" width="50%">

$$
f(x) = w_L\sigma(w_{L-1}\sigma(\cdots(w_2*\sigma(w_1*x+b_1)+b_2))+b_{L-1})+b_L
$$

Mostly used in supervised learning, with loss function and back propagation. ([A useful tool for plotting the schematic of network](http://alexlenail.me/NN-SVG/index.html))

When doing classification tasks, we usually use crossentropy loss:

$$
\mathcal L(x_n,\theta) = -\log(\frac{\exp(o_{x_n,y_n})}{\sum_{c=1}^{C}\exp(o_{x_n,c})})
$$

In [ ]:
#Construct the network
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(30, 50),
            nn.ReLU(inplace = True),
            nn.Linear(50, 2),
        )

    def forward(self, x):                             
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
    
breast_cancer = sklearn.datasets.load_breast_cancer()  
X = breast_cancer.data
Y = breast_cancer.target

# Prepocessing of the data
for i in range(X.shape[1]):
    X[:,i] = X[:,i]/max(X[:,i])

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.1, stratify=Y, random_state=0)

train_dataset = torch.from_numpy(X_train)
valid_dataset = torch.from_numpy(X_test)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=X_train.shape[0], shuffle=False)
valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset, batch_size=X_test.shape[0])
Y_train = torch.from_numpy(Y_train)
Y_test = torch.from_numpy(Y_test)

model = SimpleNet()
print(model)

In [ ]:
# Training the network
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
train_accuracy = []
test_accuracy = []
best_prec = 0
min_loss = 1

for epoch in range(0, 20000):

    model.train()
    # train for one epoch
    for i, input in enumerate(train_loader):
                    
        # compute output
        output = model(input.to(torch.float32))
        loss = criterion(output, Y_train)

    # print training accuracy and test accuracy after every 100 epochs.
        if epoch % 200 == 0:

            train_total = 0
            train_correct = 0
            train_loss = 0

            _, predicted = torch.max(output.data, 1)
            train_total =output.shape[0]
            train_correct = (predicted == Y_train).sum().item()
            prec = train_correct / train_total

            train_accuracy.append(prec)
            print('Epoch [{}/{}], Loss: {:.5f}, Train_Acc:{:.2f}%'.format(epoch, 20000, loss, prec*100))


        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward() 
        optimizer.step()
        
    if (epoch+1) % 200 == 0:
        
        model.eval()
        valid_correct = 0
        valid_total = 0
        with torch.no_grad():
            total_loss = 0
            for j, input in enumerate(valid_loader):
                output = model(input.to(torch.float32))

                _, predicted = torch.max(output.data, 1)
                valid_total = output.shape[0]
                valid_correct = (predicted == Y_test).sum().item()

                prec = valid_correct / valid_total
                print('Accuary on test images:{:.2f}%, loss:{:.2f}'.format(prec*100, loss))
                test_accuracy.append(prec)
                best_prec = max(prec, best_prec)

print('Best accuracy is: {:.2f}%'.format(best_prec*100))

plt.figure(figsize=(3.5,2))
plt.plot(200*torch.arange(len(test_accuracy)), test_accuracy,linewidth=2)
plt.xlabel('# epochs')
plt.ylabel('Test accuracy')
plt.show()

---
# Summary

- In this tutorial, we learned several famous brain inspired models and algorithms in terms of their basic idea and how to use them in a specific task.
- MP model can be seen as the basic units in the modern artificial neural network. Although it is simple and only possess binary input and output, it can do some simple tasks such as binary classification.
- Hebb's rule are based on the biological fact of synaptic plasticity rule, it is an algorithm for unsupervised learning, which can recognize the structure in the data.
- Hopfield model is an abstract model of memory retrieval. After a cue with a partial overlap with one of the stored memory patterns is presented, the memory item is retrieved. Because the Hopfield model has symmetric synaptic connections, memory retrieval can be visualized as downhill movement in an energy landscape.
- We also learned how to use pytorch to construct an artificial neural network, and use this network to do classification tasks.

---